In [2]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
# from pandas.core.common import SettingWithCopyWarning
import warnings
import plotly.graph_objects as go
import numpy as np
from functions import evaluateDept, combine_columns, createIndicators, find_outliers_IQR, plotCorrMatrix, fixDates, oosDetection, plotTS, outlierCorrections, plotCorrMatrix

# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings("ignore")


## Data Preparation

#### Load Original Data

In [3]:
df = pd.read_csv('walmart_cleaned.csv')
df

,Unnamed: 0,Store,Date,IsHoliday,Dept,Weekly_Sales,Next week,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,05-02-10,0,1,24924.50,46039.49,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,3,151315
1,1,1,05-02-10,0,26,11737.12,10050.92,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,3,151315
2,2,1,05-02-10,0,17,13223.76,13403.66,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,3,151315
3,3,1,05-02-10,0,45,37.44,NaN,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,3,151315
4,4,1,05-02-10,0,28,1085.29,NaN,42.31,2.572,0.00,0.00,0.0,0.00,0.00,211.096358,8.106,3,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,423281,45,26-10-12,0,13,26240.14,NaN,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,2,118221
421566,423282,45,26-10-12,0,16,2660.02,NaN,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,2,118221
421567,423283,45,26-10-12,0,32,4131.54,NaN,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,2,118221
421568,423284,45,26-10-12,0,83,717.82,NaN,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,2,118221


#### Look into the available timeseries

In [4]:
evaluateDept(df)

#### Select 3 Depts

The selected Depts are No3, No14 and No16

In [5]:
# Check the selected Depts across all stores
evaluateDept(df[((df['Dept']==3) | (df['Dept']==14) | (df['Dept']==16))])

In [6]:
# Check the selected Depts in the selected store
evaluateDept(df[((df['Dept']==3) | (df['Dept']==14) | (df['Dept']==16)) & (df['Store']==30)])

#### Clean up the selected data

In [7]:
df = pd.read_csv('walmart_cleaned.csv')

df['Date'] = pd.to_datetime(df['Date'])

# Keep only the selected depts and store.
df = df[((df['Dept']==3) | (df['Dept']==14) | (df['Dept']==16)) & (df['Store']==30)]

# As we are dealing with only one store we can drop this column, as well as the first column that does not contain any info.
df = df.drop(columns=['Unnamed: 0', 'Store'])

# Rename some columns for better understanding.
df = df.rename(columns={'Weekly_Sales':'sales', 'Store':'str', 'Dept': 'dept', 'Date': 'week'})

# Keep columns with more than one unique value
# In our scenario Type and Size is constant, so this will not benefit the model later,
# so we can drop them.
to_drop = df.columns[df.nunique() == 1]
print(f'Drop following columns: {to_drop.values}')
df = df.drop(columns=to_drop)
print('-----------------')
# Check for NaN values
# In our scenario there are no other missing values than the next week column,
# and this is expected, so nothing to fix here.
missing_values = df.isna().sum()
print('Missing Values:')
print(missing_values)
print('-----------------')

# Reindex columns for consistency.
df = df[['week', 'dept', 'sales', 'IsHoliday', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Next week']]

df

Drop following columns: ['Type' 'Size']
-----------------
Missing Values:
week              0
IsHoliday         0
dept              0
sales             0
Next week       429
Temperature       0
Fuel_Price        0
MarkDown1         0
MarkDown2         0
MarkDown3         0
MarkDown4         0
MarkDown5         0
CPI               0
Unemployment      0
dtype: int64
-----------------


,week,dept,sales,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Temperature,Fuel_Price,CPI,Unemployment,Next week
286551,2010-05-02,16,974.31,0,0.00,0.00,0.00,0.0,0.00,39.05,2.572,210.752605,8.324,NaN
286569,2010-05-02,14,1134.75,0,0.00,0.00,0.00,0.0,0.00,39.05,2.572,210.752605,8.324,NaN
286595,2010-05-02,3,1052.98,0,0.00,0.00,0.00,0.0,0.00,39.05,2.572,210.752605,8.324,NaN
286625,2010-12-02,14,1310.84,1,0.00,0.00,0.00,0.0,0.00,37.77,2.548,210.897994,8.324,NaN
286633,2010-12-02,3,983.75,1,0.00,0.00,0.00,0.0,0.00,37.77,2.548,210.897994,8.324,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293651,2012-10-19,3,708.15,0,37.11,0.00,0.15,0.0,1030.33,68.52,3.594,223.059808,6.170,NaN
293654,2012-10-19,16,1410.51,0,37.11,0.00,0.15,0.0,1030.33,68.52,3.594,223.059808,6.170,NaN
293673,2012-10-26,14,703.87,0,150.46,1.75,0.00,0.0,367.83,70.50,3.506,223.078337,6.170,NaN
293694,2012-10-26,16,1400.31,0,150.46,1.75,0.00,0.0,367.83,70.50,3.506,223.078337,6.170,NaN


##### Sales plot for the three selected depts

In [8]:
tmp = df[['week', 'dept', 'sales', 'IsHoliday', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']]

tmp['holInd'] = tmp['IsHoliday'].apply(lambda x: 0 if x == 0 else 1)  # Redundant, but to follow the rest of the code.      
    
tmp = createIndicators(tmp, ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])
tmp = tmp.sort_values(by=['week','dept'])
display(tmp)

fig = px.line(tmp, x='week', y='sales', color='dept',
             title='Sales by Week and Department', labels={'week':'Week','sales':'Sales'},
            #  template='plotly_dark',
             hover_name='dept')

fig.add_bar(x=tmp[tmp['holInd'] == 1]['week'], y=tmp[tmp['holInd'] == 1]['sales'], name='Holiday')
fig.add_bar(x=tmp[tmp['promoInd1'] == 1]['week'], y=tmp[tmp['promoInd1'] == 1]['sales'], name='Promo1')
fig.add_bar(x=tmp[tmp['promoInd2'] == 1]['week'], y=tmp[tmp['promoInd2'] == 1]['sales'], name='Promo2')
fig.add_bar(x=tmp[tmp['promoInd3'] == 1]['week'], y=tmp[tmp['promoInd3'] == 1]['sales'], name='Promo3')
fig.add_bar(x=tmp[tmp['promoInd4'] == 1]['week'], y=tmp[tmp['promoInd4'] == 1]['sales'], name='Promo4')
fig.add_bar(x=tmp[tmp['promoInd5'] == 1]['week'], y=tmp[tmp['promoInd5'] == 1]['sales'], name='Promo5')

fig.show()


,week,dept,sales,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,holInd,promoInd1,promoInd2,promoInd3,promoInd4,promoInd5
288255,2010-01-10,3,755.67,0,0.00,0.0,0.00,0.0,0.00,0,0,0,0,0,0
288231,2010-01-10,14,800.17,0,0.00,0.0,0.00,0.0,0.00,0,0,0,0,0,0
288243,2010-01-10,16,1269.97,0,0.00,0.0,0.00,0.0,0.00,0,0,0,0,0,0
286964,2010-02-04,3,827.23,0,0.00,0.0,0.00,0.0,0.00,0,0,0,0,0,0
286974,2010-02-04,14,938.75,0,0.00,0.0,0.00,0.0,0.00,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292473,2012-11-05,14,1070.77,0,701.89,0.0,2.28,0.0,1487.23,0,1,0,1,0,1
292487,2012-11-05,16,1717.52,0,701.89,0.0,2.28,0.0,1487.23,0,1,0,1,0,1
293602,2012-12-10,3,553.78,0,15.88,0.0,5.20,0.0,3374.40,0,1,0,1,0,1
293596,2012-12-10,14,876.21,0,15.88,0.0,5.20,0.0,3374.40,0,1,0,1,0,1


In [9]:
# df[df['dept']==3].to_csv('Dept03_Sales_OG.csv')
# df[df['dept']==14].to_csv('Dept14_Sales_OG.csv')
# df[df['dept']==16].to_csv('Dept16_Sales_OG.csv')

#### Preprocessing per TimeSerie

##### Plot the sales with all indicators

In [35]:
deptId = 'Dept03'
path = 'Data/' + deptId + '/'

In [37]:
timeSeries3 = pd.read_csv(path + deptId +"_Sales_OG.csv")
timeSeries3 = timeSeries3.rename(columns={'week':'date'})
timeSeries3["date"] = pd.to_datetime(timeSeries3["date"])
timeSeries3 = timeSeries3.sort_values(by=['date','dept'])

aggregationMethods = {'sales':'sum',
                    'dept':'min', 
                    'IsHoliday':'sum',
                    'Temperature': 'mean',
                    'Fuel_Price': 'mean',
                    'CPI': 'mean',
                    'Unemployment': 'mean',
                    'MarkDown1': 'mean',
                    'MarkDown2': 'mean',
                    'MarkDown3': 'mean',
                    'MarkDown4': 'mean',
                    'MarkDown5': 'mean'}

timeSeries3 = fixDates(timeSeries3,aggregationMethods)

timeSeries3 = oosDetection(timeSeries3)

plotTS(timeSeries3, ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], 'IsHoliday', 'Sales by Week')

##### Find outliers and create a box plot

In [38]:
outliers = find_outliers_IQR(timeSeries3["sales"])
print("number of outliers: " + str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

fig = px.box(timeSeries3, y="sales")

fig.show()

number of outliers: 11
max outlier value: 3048.66
min outlier value: 1448.8


##### Correct Outliers

In [39]:
timeSeries3['corrected_sales'] = outlierCorrections(timeSeries3)

plotTS(timeSeries3, ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], 'IsHoliday', 'Sales by Week', ['sales', 'corrected_sales'])

##### Plot Correlation Matrix for Promos and Holidays

In [40]:
# plotCorrMatrix(timeSeries3[['corrected_sales', 'Temperature', 'Fuel_Price', 'oosInd', 'holInd', 'promoInd1', 'promoInd2','promoInd3', 'promoInd4', 'promoInd5']])
plotCorrMatrix(timeSeries3[['promoInd1', 'promoInd2','promoInd3', 'promoInd4', 'promoInd5', 'holInd']])

In [41]:
to_drop = ['dept', 'IsHoliday']
timeSeries3 = timeSeries3.drop(columns=to_drop)
timeSeries3

,week,sales,Temperature,Fuel_Price,CPI,Unemployment,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,oosInd,holInd,promoInd1,promoInd2,promoInd3,promoInd4,promoInd5,corrected_sales
0,2010-01-04,755.67,70.280,2.603,211.329874,8.1630,0.00,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,755.670000
1,2010-01-11,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,755.670000
2,2010-01-18,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,755.670000
3,2010-01-25,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,755.670000
4,2010-02-01,1345.61,73.205,2.694,210.680130,8.1495,0.00,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1345.610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2012-11-12,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,732.995961
150,2012-11-19,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,732.995961
151,2012-11-26,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,732.995961
152,2012-12-03,0.00,0.000,0.000,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,732.995961


In [32]:
timeSeries3 = timeSeries3.sort_values(by=['week'])
timeSeries3[:104].to_csv(path + '/Train_2Y.csv', index=False)
timeSeries3[104:].to_csv(path + '/Test_1Y.csv', index=False)

# timeSeries3[:140].to_csv(path + '/Train_2YPlus.csv', index=False)
# timeSeries3[140:].to_csv(path + '/Test_3Months.csv', index=False)